In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
print os.listdir('drive')

# import sys
# sys.path.append('drive/places_fc2')
# sys.path.append('drive/cnn')

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
['.Trash', 'capgen', 'imgcap', 'siamese', 'Colab Notebooks', 'GupShup', 'Music Recommender', 'Distilled AI', 'springerformat.doc', 'Untitled document.odt', 'Computer Graphics', 'ComputerGraphics_Programs.zip', 'Friends Season 6', 'F.R.I.E.N.D.S season -5', 'Friends Season 4', 'Copy of Dataset.tar.gz', 'TV', 'F

In [2]:
!pip install keras

    100% |████████████████████████████████| 327kB 1.3MB/s 


In [3]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs


WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16(include_top=True, weights='imagenet',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=1000):
    """Instantiates the VGG16 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                   )
        else:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    model.layers.pop() # Get rid of the classification layer
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    return model


Using TensorFlow backend.


In [0]:
import cPickle as pickle
from keras.preprocessing import image
#from vgg16 import VGG16
import numpy as np 
from keras.applications.imagenet_utils import preprocess_input	

counter = 0

def load_image(path):
    img = image.load_img(path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return np.asarray(x)

def load_encoding_model():
	model = VGG16(weights='imagenet', include_top=True, input_shape = (224, 224, 3))
	return model

def get_encoding(model, img):
	global counter
	counter += 1
	image = load_image('drive/capgen/Flicker8k_Dataset/'+str(img))
	pred = model.predict(image)
	pred = np.reshape(pred, pred.shape[1])
	print ("Encoding image:" + str(counter))
	print (pred.shape)
	return pred

def prepare_dataset(no_imgs = -1):
	f_train_images = open('drive/capgen/Flickr8k_text/chaljaa.txt','rb')
	train_imgs = f_train_images.read().strip().split('\n') if no_imgs == -1 else f_train_images.read().strip().split('\n')[:no_imgs]
	f_train_images.close()

	f_test_images = open('drive/capgen/Flickr8k_text/Flickr_8k.testImages.txt','rb')
	test_imgs = f_test_images.read().strip().split('\n') if no_imgs == -1 else f_test_images.read().strip().split('\n')[:no_imgs]
	f_test_images.close()

	f_train_dataset = open('drive/capgen/Flickr8k_text/flickr_8k_train_dataset.txt','wb')
	f_train_dataset.write("image_id\tcaptions\n")

	f_test_dataset = open('drive/capgen/Flickr8k_text/flickr_8k_test_dataset.txt','wb')
	f_test_dataset.write("image_id\tcaptions\n")

	f_captions = open('drive/capgen/Flickr8k_text/Flickr8k.token.txt', 'rb')
	captions = f_captions.read().strip().split('\n')
	data = {}
	for row in captions:
		row = row.split("\t")
		row[0] = row[0][:len(row[0])-2]
		try:
			data[row[0]].append(row[1])
		except:
			data[row[0]] = [row[1]]
	f_captions.close()

	encoded_images = {}
	encoding_model = load_encoding_model()

	c_train = 0
	for img in train_imgs:
		encoded_images[img] = get_encoding(encoding_model, img)
		for capt in data[img]:
			caption = "<start> "+capt+" <end>"
			f_train_dataset.write(img+"\t"+caption+"\n")
			f_train_dataset.flush()
			c_train += 1
	f_train_dataset.close()

	c_test = 0
	for img in test_imgs:
		encoded_images[img] = get_encoding(encoding_model, img)
		for capt in data[img]:
			caption = "<start> "+capt+" <end>"
			f_test_dataset.write(img+"\t"+caption+"\n")
			f_test_dataset.flush()
			c_test += 1
	f_test_dataset.close()
	with open( "drive/capgen/encoded_images.p", "wb" ) as pickle_f:
		pickle.dump( encoded_images, pickle_f )  
	return [c_train, c_test]


In [5]:
c_train, c_test = prepare_dataset()

553476096/553467096 [==============================] - 6s 0us/step
Encoding image:1
(4096,)
Encoding image:2
(4096,)
Encoding image:3
(4096,)
Encoding image:4
(4096,)
Encoding image:5
(4096,)
Encoding image:6
(4096,)
Encoding image:7
(4096,)
Encoding image:8
(4096,)
Encoding image:9
(4096,)
Encoding image:10
(4096,)
Encoding image:11
(4096,)
Encoding image:12
(4096,)
Encoding image:13
(4096,)
Encoding image:14
(4096,)
Encoding image:15
(4096,)
Encoding image:16
(4096,)
Encoding image:17
(4096,)
Encoding image:18
(4096,)
Encoding image:19
(4096,)
Encoding image:20
(4096,)
Encoding image:21
(4096,)
Encoding image:22
(4096,)
Encoding image:23
(4096,)
Encoding image:24
(4096,)
Encoding image:25
(4096,)
Encoding image:26
(4096,)
Encoding image:27
(4096,)
Encoding image:28
(4096,)
Encoding image:29
(4096,)
Encoding image:30
(4096,)
Encoding image:31
(4096,)
Encoding image:32
(4096,)
Encoding image:33
(4096,)
Encoding image:34
(4096,)
Encoding image:35
(4096,)
Encoding image:36
(4096,)
Encodi

Encoding image:64
(4096,)
Encoding image:65
(4096,)
Encoding image:66
(4096,)
Encoding image:67
(4096,)
Encoding image:68
(4096,)
Encoding image:69
(4096,)
Encoding image:70
(4096,)
Encoding image:71
(4096,)
Encoding image:72
(4096,)
Encoding image:73
(4096,)
Encoding image:74
(4096,)
Encoding image:75
(4096,)
Encoding image:76
(4096,)
Encoding image:77
(4096,)
Encoding image:78
(4096,)
Encoding image:79
(4096,)
Encoding image:80
(4096,)
Encoding image:81
(4096,)
Encoding image:82
(4096,)
Encoding image:83
(4096,)
Encoding image:84
(4096,)
Encoding image:85
(4096,)
Encoding image:86
(4096,)
Encoding image:87
(4096,)
Encoding image:88
(4096,)
Encoding image:89
(4096,)
Encoding image:90
(4096,)
Encoding image:91
(4096,)
Encoding image:92
(4096,)
Encoding image:93
(4096,)
Encoding image:94
(4096,)
Encoding image:95
(4096,)
Encoding image:96
(4096,)
Encoding image:97
(4096,)
Encoding image:98
(4096,)
Encoding image:99
(4096,)
Encoding image:100
(4096,)
Encoding image:101
(4096,)
Encoding i

Encoding image:150
(4096,)
Encoding image:151
(4096,)
Encoding image:152
(4096,)
Encoding image:153
(4096,)
Encoding image:154
(4096,)
Encoding image:155
(4096,)
Encoding image:156
(4096,)
Encoding image:157
(4096,)
Encoding image:158
(4096,)
Encoding image:159
(4096,)
Encoding image:160
(4096,)
Encoding image:161
(4096,)
Encoding image:162
(4096,)
Encoding image:163
(4096,)
Encoding image:164
(4096,)
Encoding image:165
(4096,)
Encoding image:166
(4096,)
Encoding image:167
(4096,)
Encoding image:168
(4096,)
Encoding image:169
(4096,)
Encoding image:170
(4096,)
Encoding image:171
(4096,)
Encoding image:172
(4096,)
Encoding image:173
(4096,)
Encoding image:174
(4096,)
Encoding image:175
(4096,)
Encoding image:176
(4096,)
Encoding image:177
(4096,)
Encoding image:178
(4096,)
Encoding image:179
(4096,)
Encoding image:180
(4096,)
Encoding image:181
(4096,)
Encoding image:182
(4096,)
Encoding image:183
(4096,)
Encoding image:184
(4096,)
Encoding image:185
(4096,)
Encoding image:186
(4096,)
E

Encoding image:235
(4096,)
Encoding image:236
(4096,)
Encoding image:237
(4096,)
Encoding image:238
(4096,)
Encoding image:239
(4096,)
Encoding image:240
(4096,)
Encoding image:241
(4096,)
Encoding image:242
(4096,)
Encoding image:243
(4096,)
Encoding image:244
(4096,)
Encoding image:245
(4096,)
Encoding image:246
(4096,)
Encoding image:247
(4096,)
Encoding image:248
(4096,)
Encoding image:249
(4096,)
Encoding image:250
(4096,)
Encoding image:251
(4096,)
Encoding image:252
(4096,)
Encoding image:253
(4096,)
Encoding image:254
(4096,)
Encoding image:255
(4096,)
Encoding image:256
(4096,)
Encoding image:257
(4096,)
Encoding image:258
(4096,)
Encoding image:259
(4096,)
Encoding image:260
(4096,)
Encoding image:261
(4096,)
Encoding image:262
(4096,)
Encoding image:263
(4096,)
Encoding image:264
(4096,)
Encoding image:265
(4096,)
Encoding image:266
(4096,)
Encoding image:267
(4096,)
Encoding image:268
(4096,)
Encoding image:269
(4096,)
Encoding image:270
(4096,)
Encoding image:271
(4096,)
E

Encoding image:320
(4096,)
Encoding image:321
(4096,)
Encoding image:322
(4096,)
Encoding image:323
(4096,)
Encoding image:324
(4096,)
Encoding image:325
(4096,)
Encoding image:326
(4096,)
Encoding image:327
(4096,)
Encoding image:328
(4096,)
Encoding image:329
(4096,)
Encoding image:330
(4096,)
Encoding image:331
(4096,)
Encoding image:332
(4096,)
Encoding image:333
(4096,)
Encoding image:334
(4096,)
Encoding image:335
(4096,)
Encoding image:336
(4096,)
Encoding image:337
(4096,)
Encoding image:338
(4096,)
Encoding image:339
(4096,)
Encoding image:340
(4096,)
Encoding image:341
(4096,)
Encoding image:342
(4096,)
Encoding image:343
(4096,)
Encoding image:344
(4096,)
Encoding image:345
(4096,)
Encoding image:346
(4096,)
Encoding image:347
(4096,)
Encoding image:348
(4096,)
Encoding image:349
(4096,)
Encoding image:350
(4096,)
Encoding image:351
(4096,)
Encoding image:352
(4096,)
Encoding image:353
(4096,)
Encoding image:354
(4096,)
Encoding image:355
(4096,)
Encoding image:356
(4096,)
E

...

Encoding image:6628
(4096,)
Encoding image:6629
(4096,)
Encoding image:6630
(4096,)
Encoding image:6631
(4096,)
Encoding image:6632
(4096,)
Encoding image:6633
(4096,)
Encoding image:6634
(4096,)
Encoding image:6635
(4096,)
Encoding image:6636
(4096,)
Encoding image:6637
(4096,)
Encoding image:6638
(4096,)
Encoding image:6639
(4096,)
Encoding image:6640
(4096,)
Encoding image:6641
(4096,)
Encoding image:6642
(4096,)
Encoding image:6643
(4096,)
Encoding image:6644
(4096,)
Encoding image:6645
(4096,)
Encoding image:6646
(4096,)
Encoding image:6647
(4096,)
Encoding image:6648
(4096,)
Encoding image:6649
(4096,)
Encoding image:6650
(4096,)
Encoding image:6651
(4096,)
Encoding image:6652
(4096,)
Encoding image:6653
(4096,)
Encoding image:6654
(4096,)
Encoding image:6655
(4096,)
Encoding image:6656
(4096,)
Encoding image:6657
(4096,)
Encoding image:6658
(4096,)
Encoding image:6659
(4096,)
Encoding image:6660
(4096,)
Encoding image:6661
(4096,)
Encoding image:6662
(4096,)
Encoding image:6663


Encoding image:6712
(4096,)
Encoding image:6713
(4096,)
Encoding image:6714
(4096,)
Encoding image:6715
(4096,)
Encoding image:6716
(4096,)
Encoding image:6717
(4096,)
Encoding image:6718
(4096,)
Encoding image:6719
(4096,)
Encoding image:6720
(4096,)
Encoding image:6721
(4096,)
Encoding image:6722
(4096,)
Encoding image:6723
(4096,)
Encoding image:6724
(4096,)
Encoding image:6725
(4096,)
Encoding image:6726
(4096,)
Encoding image:6727
(4096,)
Encoding image:6728
(4096,)
Encoding image:6729
(4096,)
Encoding image:6730
(4096,)
Encoding image:6731
(4096,)
Encoding image:6732
(4096,)
Encoding image:6733
(4096,)
Encoding image:6734
(4096,)
Encoding image:6735
(4096,)
Encoding image:6736
(4096,)
Encoding image:6737
(4096,)
Encoding image:6738
(4096,)
Encoding image:6739
(4096,)
Encoding image:6740
(4096,)
Encoding image:6741
(4096,)
Encoding image:6742
(4096,)
Encoding image:6743
(4096,)
Encoding image:6744
(4096,)
Encoding image:6745
(4096,)
Encoding image:6746
(4096,)
Encoding image:6747


Encoding image:6796
(4096,)
Encoding image:6797
(4096,)
Encoding image:6798
(4096,)
Encoding image:6799
(4096,)
Encoding image:6800
(4096,)
Encoding image:6801
(4096,)
Encoding image:6802
(4096,)
Encoding image:6803
(4096,)
Encoding image:6804
(4096,)
Encoding image:6805
(4096,)
Encoding image:6806
(4096,)
Encoding image:6807
(4096,)
Encoding image:6808
(4096,)
Encoding image:6809
(4096,)
Encoding image:6810
(4096,)
Encoding image:6811
(4096,)
Encoding image:6812
(4096,)
Encoding image:6813
(4096,)
Encoding image:6814
(4096,)
Encoding image:6815
(4096,)
Encoding image:6816
(4096,)
Encoding image:6817
(4096,)
Encoding image:6818
(4096,)
Encoding image:6819
(4096,)
Encoding image:6820
(4096,)
Encoding image:6821
(4096,)
Encoding image:6822
(4096,)
Encoding image:6823
(4096,)
Encoding image:6824
(4096,)
Encoding image:6825
(4096,)
Encoding image:6826
(4096,)
Encoding image:6827
(4096,)
Encoding image:6828
(4096,)
Encoding image:6829
(4096,)
Encoding image:6830
(4096,)
Encoding image:6831


Encoding image:6880
(4096,)
Encoding image:6881
(4096,)
Encoding image:6882
(4096,)
Encoding image:6883
(4096,)
Encoding image:6884
(4096,)
Encoding image:6885
(4096,)
Encoding image:6886
(4096,)
Encoding image:6887
(4096,)
Encoding image:6888
(4096,)
Encoding image:6889
(4096,)
Encoding image:6890
(4096,)
Encoding image:6891
(4096,)
Encoding image:6892
(4096,)
Encoding image:6893
(4096,)
Encoding image:6894
(4096,)
Encoding image:6895
(4096,)
Encoding image:6896
(4096,)
Encoding image:6897
(4096,)
Encoding image:6898
(4096,)
Encoding image:6899
(4096,)
Encoding image:6900
(4096,)
Encoding image:6901
(4096,)
Encoding image:6902
(4096,)
Encoding image:6903
(4096,)
Encoding image:6904
(4096,)
Encoding image:6905
(4096,)
Encoding image:6906
(4096,)
Encoding image:6907
(4096,)
Encoding image:6908
(4096,)
Encoding image:6909
(4096,)
Encoding image:6910
(4096,)
Encoding image:6911
(4096,)
Encoding image:6912
(4096,)
Encoding image:6913
(4096,)
Encoding image:6914
(4096,)
Encoding image:6915


Encoding image:6964
(4096,)
Encoding image:6965
(4096,)
Encoding image:6966
(4096,)
Encoding image:6967
(4096,)
Encoding image:6968
(4096,)
Encoding image:6969
(4096,)
Encoding image:6970
(4096,)
Encoding image:6971
(4096,)
Encoding image:6972
(4096,)
Encoding image:6973
(4096,)
Encoding image:6974
(4096,)
Encoding image:6975
(4096,)
Encoding image:6976
(4096,)
Encoding image:6977
(4096,)
Encoding image:6978
(4096,)
Encoding image:6979
(4096,)
Encoding image:6980
(4096,)
Encoding image:6981
(4096,)
Encoding image:6982
(4096,)
Encoding image:6983
(4096,)
Encoding image:6984
(4096,)
Encoding image:6985
(4096,)
Encoding image:6986
(4096,)
Encoding image:6987
(4096,)
Encoding image:6988
(4096,)
Encoding image:6989
(4096,)
Encoding image:6990
(4096,)
Encoding image:6991
(4096,)
Encoding image:6992
(4096,)
Encoding image:6993
(4096,)
Encoding image:6994
(4096,)
Encoding image:6995
(4096,)
Encoding image:6996
(4096,)
Encoding image:6997
(4096,)
Encoding image:6998
(4096,)
Encoding image:6999


In [0]:
import os
os.listdir('drive/capgen/Flickr8k_text')

['Flickr_8k.trainImages.txt',
 'Flickr8k.token.txt',
 'CrowdFlowerAnnotations.txt',
 'Flickr8k.lemma.token.txt',
 'Flickr_8k.devImages.txt',
 'ExpertAnnotations.txt',
 'Flickr_8k.testImages.txt',
 'readme.txt',
 'README.md']

In [0]:
x=open('drive/capgen/Flickr8k_text/chaljaa.txt','rb')